# Easily generate initial states with Overlapper: Block2 and Dice

This notebook demonstrates how to use Overlapper to easily execute the density-matrix renormalization group method (DMRG) implemented in Block2, and semistochastic heat-bath configuration interaction (SHCI) calculations implemented in Dice -- and convert their outputs to generate initial states for quantum algorithms. 

Warning: for DMRG, you must have the Block2 library installed to run this example. It can be installed with 

```pip install block2```

For SHCI, you must have the Dice library installed. The installation involves some subtleties: we suggest installation and configuration instructions in README, but this may vary depending on your machine and environment setup.

## **Use Overlapper to run quantum chemistry calculations in Block2**

To simplify the process of creating initial states from diverse outputs, Overlapper implements wrappers around a wide variety of quantum chemistry methods. This notebook discusses the DMRG method as implemented in the Block2 library.

As before, start by creating a `Molecule` in PySCF

In [1]:
from pyscf import gto
mol = gto.M(atom=[["Be", (0,0,0)], ["Be", (1.1,0,0)]], basis='sto3g', spin=0)

Use Overlapper to execute a Hartree-Fock calculation on the molecule

In [2]:
from overlapper.state import do_hf, hf_state
hftype = "rhf"
TOL = 1e-1

# run an HF calculation
hf, hf_e, hf_ss, hf_sz = do_hf(mol, hftype)
wf_hf = hf_state(hf)
print(f"HF wavefunction: \n {wf_hf}")
print(f"\nRHF energy: {hf_e[0]:.3f}")
print(f"\nRHF S^2 and Sz: {hf_ss[0]:.3f}, {hf_sz[0]:.3f}")

/home/chiffa/miniconda3/envs/overlapper_v0.0.1/lib/python3.9/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


HF wavefunction: 
 {(15, 15): 1.0}

RHF energy: -28.336

RHF S^2 and Sz: 0.000, 0.000


For future reference, compute the exact solution

In [3]:
from overlapper.state import do_casci, casci_state
ncas, nelecas = 10, (4,4)
mycasci, mycasci_e, mycasci_ss, mycasci_sz = do_casci(hf, ncas, nelecas)
wf_casci = casci_state(mycasci, tol=TOL)
print(wf_casci)
print(f"\nCASCI energy: {mycasci_e[0]:.3f}")
print(f"\nCASCI spins: {mycasci_ss[0]:.3f}, {mycasci_sz[0]:.3f}")

{(15, 23): 0.6855938134877609, (23, 15): 0.685641162009288}

CASCI energy: -28.396

CASCI spins: 0.000, 0.000


Running DMRG requires defining more parameters than for most other methods. We assume the user is familiar with the basics of DMRG and of Block2. To execute such a calculation using Overlapper, we again use a `do_xxx()` method. The key things to specify are: the schedule -- prescribing the bond dimensions, number of sweeps, noise and Davidson tolerance to be used during the execution; as well as the active space of the model.

All the same parameters as for the other methods, such as `nroots` for additional states and `verbose` are also available. Advanced users may choose to take advantage of further tunability by varying other keyword arguments: for a full list, see the documentation.

An important parameter to mention is `mpssym`, which specifies whether the DMRG calculation is executed in SZ symmetric mode (default) or in SU(2) symmetric mode. 

In [4]:
from overlapper.state import do_dmrg, dmrg_state
workdir = "dmrg_temp"
bond_dims = [10,50,100,200]
n_sweeps = [20,20,20,20]
noises = [1e-4,1e-4,1e-5,1e-5]
thrds = [1e-5,1e-5,1e-6,1e-6]
sch = [bond_dims, n_sweeps, noises, thrds]
ncas, nelecas = 10, (4,4)
mydmrg, mydmrg_e, mydmrg_ss, mydmrg_sz = do_dmrg(hf, ncas, nelecas, sch, nroots=3, n_threads=6,\
                                    workdir=workdir, reorder=None, smp_tol=TOL, verbose=0)
# extract parameters for state construction
wf_dmrg1 = dmrg_state(mydmrg, state=0, tol=TOL)
wf_dmrg2 = dmrg_state(mydmrg, state=1, tol=TOL)
wf_dmrg3 = dmrg_state(mydmrg, state=2, tol=TOL)
print(wf_dmrg1)
print(wf_dmrg2)
print(wf_dmrg3)
print(f"\nDMRG energy: {mydmrg_e}")
print(f"\nDMRG spins: {mydmrg_ss}, {mydmrg_sz}")

{(15, 23): 0.670968834498929, (23, 15): -0.6709702900090161, (27, 71): -0.11355753045295006, (71, 27): 0.11355773631839361}
{(15, 15): -0.6340618250007888, (23, 23): -0.6340523855425322, (39, 39): 0.1702076090944374, (27, 27): 0.3517630085769345}
{(15, 15): 0.6856050329928892, (23, 23): -0.6856305528973268}

DMRG energy: [-28.43937868 -28.40842185 -28.39605309]

DMRG spins: [2.00000014e+00 2.10540266e-06 8.85601416e-07], [0 0 0]


Unlike CASCI, DMRG in SZ symmetric mode does not control the total value of spin. By computing the lowest three states, we see that two of the lowest-energy states in DMRG in fact have finite total spin -- we recognize these states from finite-spin CASCI calculations. Only the third one aligns with the ground-state found in CASCI: comparing the wavefunctions, we see that they are identical.

Unlike for PySCF-bound methods, the first item returned by `do_dmrg()` is actually a tuple of Slater determinants and coefficients, and not a solver-type object. However, it is still processed by the `dmrg_state()` method to yield a wavefunction in precisely the same way as for all other methods. Under the hood, the MPS is reconstructed into a series of Slater determinants, with those whose coefficients are below `smp_tol` being neglected.

One final crucial note: the `do_dmrg()` method in Overlapper is special, on account of the fact that DMRG is used for more than initial state generation -- in particular, for the computation of moments, and for running the resolvent method of energy distribution approximation. The special option `return_objects=True` changes the return type from standard initial state one to returning three Block2 objects: `mpo`, the matrix-product operator of the Hamiltonian; `mps`, the matrix-product format of the wavefunction; and `driver`, the Block2 Solver-type object.

## **Use Overlapper to run quantum chemistry calculations in Dice**

Assuming Dice is installed, its output is among the easiest to convert to the standard Overlapper form. Its execution requires somewhat fewer parameters than that of the DMRG method: we again assume a level of familiarity with the Dice and SHCI approach. 

Key arguments to mention are `eps1_arr`, which specifies a list of cutoffs to use for the construction of the variational space; and `output_file`, which specifies the location where the output from the Dice executable is written. The output file is crucial because it contains the information about the wavefunction, which is read internally by Overlapper and returned as a tuple of determinants and coefficients, similar to the DMRG case.

One final keyword argument of note is `dets_budget`: set by default to 1000, it controls how many determinants (and their coefficients) are printed by Dice to disk and then captured by Overlapper. Notice that this (and spin control) will only work for you if you follow our specific procedure for installing and modifying Dice.

In [5]:
from overlapper.state import do_shci, shci_state
from overlapper.utils.shci_utils import cleanup_dice_files
output_file = "SHCI_output.dat"
ncas, nelecas = 10, (4,4)
eps1_arr = [1e-2,1e-3,1e-4]
n_sweeps = [20,20,20]
myshci, myshci_e, myshci_ss, myshci_sz = do_shci(hf, ncas, nelecas, \
                            eps1_arr, n_sweeps, output_file, nroots=3)
wf_shci1 = shci_state(myshci, state=0, tol=TOL)
wf_shci2 = shci_state(myshci, state=1, tol=TOL)
wf_shci3 = shci_state(myshci, state=2, tol=TOL)
print(wf_shci1)
print(wf_shci2)
print(wf_shci3)
cleanup_dice_files(".")
print(f"\nSHCI energy: {myshci_e}")
print(f"\nSHCI spins: {myshci_ss}, {myshci_sz}")

{(23, 23): 0.6340407461, (15, 15): 0.6340404937, (27, 27): -0.3517652859, (39, 39): -0.1702668749}
{(23, 23): -0.6856180695, (15, 15): 0.6856178331}
{(39, 39): -0.7825510335, (71, 71): 0.3804894979, (51, 51): 0.2460486137, (43, 43): 0.2460483374, (99, 39): -0.1123359442, (39, 99): -0.1123359442, (15, 15): -0.1007252839, (23, 23): -0.1007244881}
Dice files cleaned up!

SHCI energy: [-28.40843583 -28.39606299 -28.27452451]

SHCI spins: [0. 0. 0.], [0. 0. 0.]


Unlike in the case of DMRG, here with SHCI we see that the states come out with total spin zero, making their identification with the corresponding CASCI states more straightforward.

While in this notebook we demonstrated the use of these methods on a simple, small molecule, these methods are very general and performant and can be straightforwardly used to generate good-quality initial states for systems with up to 50-70 orbitals.